# Hidden Markov Model (HMM) Based Music Generator

### Introduction

In this notebook, we will be generating Music (no vocals though) using an HMM via [Baum-Welch](https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm) Training Algorithm.

In the context of Music Generation, the states might represent underlying musical concepts (like note pitches or chord types), and observations could be specific notes or chords played at a given time. Hence, generating music involves moving through the states based on transition probabilities and producing musical notes based on the emission probabilities associated with each state. The emission probabilities dictate how likely it is to observe each possible note or chord (observation symbol) when in a given state.

## Terminology

__Baum Welch__: is an __*Unsupervised*__ training algorithm that involves adjusting the HMM's parameters (transition, emission, and initial state probabilities) to best account for the observed sequences.The training process involves:
- Expectation step (E-step): Estimate the likely sequences of hidden states (could be something implicit like musical concepts like chords or rhythms) given the current parameters of the model and the observed data.
- Maximization step (M-step): Update the model's parameters to maximize the likelihood of the observed data, based on the estimated sequences of hidden states.

![Baum Welch](unsupervised_learning.png)

## Resources

For additional details of the working of Baum-Welch Training you can consult these medium articles [Baum-Welch algorithm](https://medium.com/mlearning-ai/baum-welch-algorithm-4d4514cf9dbe) and [Baum-Welch algorithm for training a Hidden Markov Model — Part 2 of the HMM series](https://medium.com/analytics-vidhya/baum-welch-algorithm-for-training-a-hidden-markov-model-part-2-of-the-hmm-series-d0e393b4fb86) as reference.

A more technical overview is covered by Rabiner in his paper on [A Tutorial on Hidden Markov Models and Selected Applications in Speech Recognition](http://www.stat.columbia.edu/~liam/teaching/neurostat-fall17/papers/hmm/rabiner.pdf).

If the above link is a bit difficult to digest, you can consult the following slides by Stanford's Dan Jurafsky in his course [LSA 352: Speech Recognition and Synthesis](https://nlp.stanford.edu/courses/lsa352/lsa352.lec7.6up.pdf).


For this notebook, we will stick to standard libraries i.e. `numpy`, `tqdm`, `hmmlearn` and `muspy`.

In [ ]:
!pip install muspy
!pip install hmmlearn

import numpy as np
from tqdm.notebook import tqdm as tqdm
import muspy
from hmmlearn.hmm import CategoricalHMM

# muspy.download_musescore_soundfont()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.4 MB/s eta 0:00:00
  Created wheel for pretty-midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592289 sha256=5ae50cb63e543a46739ed0939e4980a9fa1d9df7a711ce319a2d598e55fb94d2
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty-midi
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 3.6 MB/s eta 0:00:00


**MusPy** is an open source Python library for symbolic music generation. It provides essential tools for developing a music generation system, including dataset management, data I/O,  
data preprocessing and model evaluation.  
**Documentation**: https://salu133445.github.io/muspy/

In [ ]:
#List of available datasets in muspy (could also add your own datasets)
#Link: https://salu133445.github.io/muspy/datasets/datasets.html
print(muspy.list_datasets())

[<class 'muspy.datasets.emopia.EMOPIADataset'>, <class 'muspy.datasets.essen.EssenFolkSongDatabase'>, <class 'muspy.datasets.haydn.HaydnOp20Dataset'>, <class 'muspy.datasets.hymnal.HymnalDataset'>, <class 'muspy.datasets.hymnal.HymnalTuneDataset'>, <class 'muspy.datasets.jsb.JSBChoralesDataset'>, <class 'muspy.datasets.lmd.LakhMIDIAlignedDataset'>, <class 'muspy.datasets.lmd.LakhMIDIDataset'>, <class 'muspy.datasets.lmd.LakhMIDIMatchedDataset'>, <class 'muspy.datasets.maestro.MAESTRODatasetV1'>, <class 'muspy.datasets.maestro.MAESTRODatasetV2'>, <class 'muspy.datasets.maestro.MAESTRODatasetV3'>, <class 'muspy.datasets.music21.Music21Dataset'>, <class 'muspy.datasets.musicnet.MusicNetDataset'>, <class 'muspy.datasets.nes.NESMusicDatabase'>, <class 'muspy.datasets.nmd.NottinghamDatabase'>, <class 'muspy.datasets.wikifonia.WikifoniaDataset'>]


Since Baum-Welch is known for its slow convergence, we'll take the lightest dataset available from muspy datasets called the __HaydnOp20__ Dataset consisting of 1.26 hours of recordings \
comprising of 24 classical songs

In [ ]:
my_dataset = muspy.datasets.HaydnOp20Dataset(root = './data', download_and_extract=True)
my_dataset = my_dataset.convert()

1114112it [00:00, 14207433.77it/s]        


Successfully downloaded source : /content/data/haydnop20v1.3_annotated.zip .
Extracting archive : /content/data/haydnop20v1.3_annotated.zip ...
Successfully extracted archive : /content/data .
Converting and saving the dataset...


100%|██████████| 24/24 [02:39<00:00,  6.65s/it]

Successfully saved 15 out of 24 files.


## Section 1: (HMM + Baum Welch From Scratch)

Muspy offers all datasets in 4 different representations as mentioned below:

![Alt text](muspy_representations.png)

Initially, we are only interested in modelling through time and to keep it simple, we'll begin with the __Pitch Representation__. More details here:

![text](muspy_to_pitch.png)

In [ ]:
music_collection = []
for music in my_dataset:
    music_collection.append(muspy.to_pitch_representation(music, use_hold_state=True))

### Singing HMM Class
The __Singing_HMM__ Class contains the following methods:

1. `__init__(self, corpus)`: initializes the __POS_HMM__ and prepares it for the parameter initialization phase, contains:
    - a corpus, consisting of unlabeled  sequences of musical units (i.e. all the music songs are flattened and concatenated)
    - a hidden_state_size (default to 10), higher values capture more variability but converge slowly.
    - a tuple of all the unique
    - a dictionary for mapping the pitches to its unique integer identifier.
    - some additional variables to reduce code redundancy in latter parts such as len()
    - Transition, Emission and Initial State Probability Matrices which are initialized to Zeros.

2. `init_mat(self, init_scheme='uniform')`:  initializes the transition, emission and probability matrices either with a 'uniform' value or values sampled randomly from a uniform distribution and normalizes the matrice row wise.

3. `forward(self, sequence)`: implements the Forward stage of the Forward-Backward Algorithm.
- Feel free to modify function signature and return values.
- Do not change the function name.
4. `backward(self, sequence)`:  implements the Forward stage of the Forward-Backward Algorithm.
- Feel free to modify function signature and return values.
- Do not change the function name.
6. `baum_welch(sequence, alpha, beta)`: implements the Baum Welch Training Algorithm.
- Feel free to modify function signature and return values.
- Do not change the function name.
7. `softmax(self, x, temperature=1.0)`: calculates the softmax of a given input x adjusting the sharpness of the distribution based on a temperature parameter.

8. `temperature_choice(self, probabilities, temperature=1.0)`: applies a temperature scaling to a set of probabilities and selects an index based on the adjusted probabilities.

9. `sample_sequence(self, length, strategy = "temperature", temperature = 1.0)`: generates a sequence of elements based on a given strategy (probabilistic or temperature) and a specified length. Strategies consists of:
* `probabilistic` strategy:
    -  Samples the initial state based on initial state probabilities.
    -  Iterates over the desired sequence length, sampling an observation based on the current state's emission probabilities, appending the observation to the sequence, and then transitioning to the next state based on the current state's transition probabilities.
* `temperature` strategy:
    -  Similar to the probabilistic strategy but applies temperature scaling to the choice of initial state, observation sampling, and state transitions to adjust the randomness of the choices.

A common error with Baum Welch Implementations is overflow error, arising from division by 0. This is due to long sequences yielding  smaller values of alpha and beta. Hence, wherever division occurs, the denominator variable (which is a result of multiplication with alpha or beta) is close to 0.

The following are some ways with which the this can be alleviated __(the hacky ways might/might not work, so be wary)__:

- Hacky way #1 (Working with smaller chunks of observed sequences): For every iteration, rather than going over the concatenated music sequences or each music sequence, you can further break down your musical sequences into even smaller chunks and go over those instead.

- Hacky way #2 (Add a small epsilon value to the denominator): Add a small episilon value like 1e-12 to the denominator wherever the division by 0 error occurs.

- Proper way #1 (The [log-sum-exp](https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/) trick): For an HMM, the smaller values can be dealt with by passing them through
    log and converting the multiplications to additions and then brought back via exponentiating them.

    - Another [intro](https://www.xarg.org/2016/06/the-log-sum-exp-trick-in-machine-learning/) for the log-sum-exp, if the previous one was unclear.
    - [Hidden Markov Models By Herman Kemper](https://www.kamperh.com/nlp817/notes/05_hmm_notes.pdf) illustrates the use of log-sum-exp technique in Baum Welch Implementation (particularly Forward and Backward Passes).
    - [Recition 7: Hidden Markov Models](https://www.cs.cmu.edu/~mgormley/courses/10601-s23/handouts/hw7_recitation_solution.pdf) gives an idea of the usage of log-sum-exp in the forward-backward algorithm.
    - This HMM github [repo](https://github.com/jwmi/HMM/blob/master/hmm.jl) has implemented the log-sum-exp trick in julia language.
    - The following [blog post](https://gregorygundersen.com/blog/2020/11/28/hmms/#implementation) might also be helpful for implementation of baum-welch using log-sum-exp trick.
    - The following paper on [Numerically Stable Hidden Markov Models](http://bozeman.genome.washington.edu/compbio/mbt599_2006/hmm_scaling_revised.pdf) gives pseudocodes for working in the log domain for the HMMs (although not necessarily the log-sum-exp trick as is).

- Proper way #2 (Scaling Factors): involves scaling the alpha and beta values to avoid underflows.
    - The following blog post explains the maths behind scaling [Scaling Factors for Hidden Markov Models](https://gregorygundersen.com/blog/2022/08/13/hmm-scaling-factors/)
    - This stackexchange post [Scaling step in Baum-Welch algorithm](https://stats.stackexchange.com/questions/274175/scaling-step-in-baum-welch-algorithm) contains two answers which can also be consulted.

__How do you know the HMM is converging?__:

Since Baum Welch algorithm guarantees convergence to the local (not global) maxima, near zero values are difficult to achieve.  \
Hence, a convergening HMM would have the log likelihoods going towards 0 (although still far from it). You can find a sample cell output  \
below showing the log likelihoods decreasing. Another way is to see is that the post-convergence generated music would be better than the  \
starting HMM (which has uniform or randomly initialized matrices).

__How do you know the HMM has converged?__:

One way is to monitor the difference between two successive log likelihoods and stop when the differences goes below a certain threshold.

In [172]:
class Singing_HMM:
    def __init__(self, corpus, hidden_state_size=10):
        self.corpus = [seq.flatten().tolist() for seq in corpus]
        self.hidden_state_size = hidden_state_size
        self.music_seq = [note for seq in self.corpus for note in seq]
        self.vocab = tuple(set(self.music_seq))
        self.vocab2index = {note: i for i, note in enumerate(self.vocab)}
        self.vocab_len = len(self.vocab)

        self.transition_mat = np.zeros((self.hidden_state_size, self.hidden_state_size))
        self.emission_mat = np.zeros((self.hidden_state_size, self.vocab_len))
        self.initial_state_prob = np.zeros(self.hidden_state_size)


    def init_mat(self, init_scheme='uniform'): # Can be optionally modified for another initialization scheme (not necessary for the assignment)
        if init_scheme == 'uniform':
            self.transition_mat = np.ones((self.hidden_state_size, self.hidden_state_size))
            self.emission_mat = np.ones((self.hidden_state_size, self.vocab_len))
            self.initial_state_prob = np.ones(self.hidden_state_size)
        elif init_scheme == 'random':
            self.transition_mat = np.random.rand(self.hidden_state_size, self.hidden_state_size)
            self.emission_mat = np.random.rand(self.hidden_state_size, self.vocab_len)
            self.initial_state_prob = np.random.rand(self.hidden_state_size)

        self.transition_mat /= self.transition_mat.sum(axis=1, keepdims=True)
        self.emission_mat /= self.emission_mat.sum(axis=1, keepdims=True)
        self.initial_state_prob /= self.initial_state_prob.sum()

    def forward(self, sequence):
        """
        Forward algorithm for calculating the probabilities of a sequence.
        """

        a = self.transition_mat
        b = self.emission_mat

        initial_distribution = self.initial_state_prob

        alpha = np.zeros((len(sequence), self.hidden_state_size))
        alpha[0, :] = initial_distribution * b[:, self.vocab2index[sequence[0]]]

        for t in range(1, len(sequence)):
            o_t = self.vocab2index[sequence[t]]
            for j in range(self.hidden_state_size):
              sum = 0
              for i in range(self.hidden_state_size):
                sum += alpha[t - 1,i] * a[i, j] * b[j, o_t]
              alpha[t, j] = sum


        return alpha


    def backward(self, sequence):
        """
        Backward algorithm for calculating the probabilities of a sequence.
        """
        a = self.transition_mat
        b = self.emission_mat
        V = sequence

        beta = np.zeros((len(V), self.hidden_state_size))
        beta[len(V) - 1] = np.ones((self.hidden_state_size))

        for t in range(len(sequence) - 2, -1 , -1):
            o_tp1 = self.vocab2index[sequence[t+1]]
            for i in range(self.hidden_state_size):
              sum = 0
              for j in range(self.hidden_state_size):
                sum += beta[t+1, j] * a[i, j] * b[j, o_tp1]
              beta[t, i] = sum

        return beta


    def e_step(self, alpha, beta, sequence):

        a = self.transition_mat
        b = self.emission_mat
        V = sequence

        gamma = np.zeros((len(V), self.hidden_state_size))
        denom = alpha[len(V) - 1, self.vocab2index[sequence[-1]]]
        for t in range(len(sequence)):
            for j in range(self.hidden_state_size):
              # print(alpha[t,j], beta[t,j], denom, (alpha[t,j] * beta[t,j])/denom )
              gamma[t,j] = (alpha[t,j] * beta[t,j]) / denom

        xi = np.zeros((len(V), self.hidden_state_size, self.hidden_state_size))


        for t in range(len(sequence) - 1):
            o_tp1 = self.vocab2index[sequence[t+1]]
            for i in range(self.hidden_state_size):
              for j in range(self.hidden_state_size):
                xi[t,i,j] = (alpha[t,i] * a[i,j] * b[j, o_tp1] * beta[t+1, j]) / denom

        return gamma, xi


    def m_step(self, gamma, xi, sequence):

      new_a = self.transition_mat
      new_b = self.emission_mat
      V = sequence

      for i in range(self.hidden_state_size):
        denom = 0
        for j in range(self.hidden_state_size):
          num = 0
          for t in range(len(sequence) - 1):
            num += xi[t,i,j]
            denom += xi[t,i,j]
          new_a[i,j] = num
        new_a[i,:] = new_a[i,:] / denom


      for j in range(self.hidden_state_size):
        for v_k in range(self.vocab_len):
          num = 0
          total = 0
          for t in range(len(sequence) - 1):
            total += gamma[t,j]
            if self.vocab2index[sequence[t]] == v_k:
              num += gamma[t,j]
          new_b[j,v_k] = num/total

      return new_a, new_b

    def baum_welch(self, n_iter=100, tol=1e-4):
        """
        Perform Baum-Welch training to update the model's parameters.
        """

        prev_log_likelihood = float('-inf')  # Initialize with negative infinity (DO NOT CHANGE THIS VARIABLE)
        M = self.transition_mat.shape[0]

        # split_music = np.array_split(self.music_seq, 100)
        split_music = [self.music_seq[i:i+100] for i in range(0, len(self.music_seq), 100)]

        print(self.music_seq)
        print(len(self.music_seq))

        print(split_music)
        print(len(split_music))
        print(len(split_music[0]))

        for iteration in tqdm(range(n_iter), desc="Training Progress", leave=True):
            log_likelihood = 0 # Log likelihood for this iteration (DO NOT CHANGE THIS VARIABLE)
            # print(len(V))

            V = split_music[n_iter]
            alpha = self.forward(V)
            beta = self.backward(V)

            log_likelihood = np.sum(np.log(alpha[-1,:]))

            gamma, xi = self.e_step(alpha, beta, V)
            new_a, new_b = self.m_step(gamma, xi, V)

            self.transition_mat = new_a
            self.emission_mat = new_b

            # print(self.transition_mat)

            if iteration == 0:
                convergence_rate = convergence_diff = np.nan  # Print nan for the first iteration
            else:
                convergence_diff = np.abs(log_likelihood - prev_log_likelihood)
                convergence_rate = convergence_diff / np.abs(prev_log_likelihood)

            #Note that Log Likelihoods would be negative and would increase (i.e. go in the direction of 0) as the model converges.
            # Log Likelihoods may be far from 0, but the increasing trend should remain present.
            tqdm.write(f"Iteration {iteration + 1}: Log Likelihood: {log_likelihood}, Convergence Difference: {convergence_diff} , Convergence Rate: {convergence_rate}")

            if iteration > 0 and convergence_rate < tol:
                tqdm.write("Convergence achieved.")
                break

            prev_log_likelihood = log_likelihood

    def softmax(self, x, temperature=1.0):
        '''Compute softmax values for each set of scores in x.'''
        e_x = np.exp((x - np.max(x)) / temperature)
        return e_x / e_x.sum()

    def temperature_choice(self, probabilities, temperature=1.0):
        '''Apply temperature to probabilities and make a choice.'''
        adjusted_probs = self.softmax(np.log(probabilities + 1e-9), temperature)  # Adding epsilon to avoid log(0)
        return np.random.choice(len(probabilities), p=adjusted_probs)

    def sample_sequence(self, length, strategy = "temperature", temperature = 1.0):
        sequence = []
        if strategy == 'probabilistic':
            # Sample the initial state
            state = np.random.choice(self.hidden_state_size, p=self.initial_state_prob)
            for _ in range(length):
                # Sample an observation (note) based on the current state
                note = np.random.choice(self.vocab, p=self.emission_mat[state])
                sequence.append(note)
                # Transition to the next state
                state = np.random.choice(self.hidden_state_size, p=self.transition_mat[state])
        elif strategy == 'temperature':
            # Sample the initial state with temperature
            state = self.temperature_choice(self.initial_state_prob, temperature)
            for _ in range(length):
                # Apply temperature to emission probabilities and sample a note
                note = self.temperature_choice(self.emission_mat[state], temperature)
                sequence.append(self.vocab[note])
                # Transition to the next state with temperature
                state = self.temperature_choice(self.transition_mat[state], temperature)
        return sequence

In [175]:
#Specify values and run the code to test your implementation
pos_hmm = Singing_HMM(corpus = music_collection, hidden_state_size = 10)
pos_hmm.init_mat(init_scheme = 'random') #Note: HMMs are sensitive to intialization schemes
pos_hmm.baum_welch(tol = 1e-4, n_iter= 100)

[70, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 39, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 72, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 68, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 67, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 58, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 68, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 65, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 63, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 39, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 62, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 60, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 58, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 58, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 58, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 58, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 58, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 1

Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Iteration 1: Log Likelihood: -3994.9271792766854, Convergence Difference: nan , Convergence Rate: nan
Iteration 2: Log Likelihood: -921.0724907429233, Convergence Difference: 3073.854688533762 , Convergence Rate: 0.7694394792673814
Iteration 3: Log Likelihood: -920.7040108519631, Convergence Difference: 0.3684798909602023 , Convergence Rate: 0.0004000552558713288
Iteration 4: Log Likelihood: -920.3248376542396, Convergence Difference: 0.3791731977235031 , Convergence Rate: 0.0004118296360766795
Iteration 5: Log Likelihood: -919.8953686143158, Convergence Difference: 0.42946903992378793 , Convergence Rate: 0.00046664940720108743
Iteration 6: Log Likelihood: -919.3809763640825, Convergence Difference: 0.5143922502332998 , Convergence Rate: 0.0005591856071719923
Iteration 7: Log Likelihood: -918.7444740245219, Convergence Difference: 0.6365023395605931 , Convergence Rate: 0.0006923161952706458
Iteration 8: Log Likelihood: -917.94389735412, Convergence Difference: 0.8005766704018242 , Conv

In [177]:
notes_seq = pos_hmm.sample_sequence(1024, strategy= "probabilistic") #Feel free to experiment with the sampling strategy
synthetic_music = muspy.from_pitch_representation(np.array(notes_seq), resolution=24, program=0, is_drum=False, use_hold_state=True, default_velocity=64)
muspy.write_midi('pitch_based.mid', synthetic_music) #Specify the path to save the MIDI file, name it "pitch_based.mid"

You can visualize your results here: https://cifkao.github.io/html-midi-player/  
  
__*P.S*__: You can use muspy.write_audio to convert the music object directly to wav file but that requires installation of a few softwares (not worth the hassle).

*You might notice that the results are although better than random but they are not as awe-inspiring as intended.
The reason being that our model is unable to capture the  \
variability of the different music styles (our dataset comprises of). However, there is a way to generate better music, that is taking a sufficiently long MIDI  \
(could be other formats as well) sound track(s) of a single artist (EDM or any music which has repetitiveness in it) and refitting your HMM.  \
The relevent function here would be muspy.read_midi().  

## Section 2: Synthetic Music Generation via __HMMLearn__

#### __Note:__ For any model that you train/fit, remember to set __verbose = True__

In [146]:
hidden_states = 50 #Number of hidden states in the HMM model (Feel free to change or experiment with this value)
sythetic_music_sequence_length = 1024 #Length of the synthetic music sequence to be generated (could be either a time step, an event or a note)

For starters, let's replicate what we did above manually with our HMM Library. Since, we already did pitch based representation,  
let's do it for **Event Based Representation** (which is essentially denotes music as a sequence of events). So while pitch based representation  
is between 0-128 unique pitch values, the event based representation is between 0-387 unique events.

In [147]:
# Write your code here by fitting and generating from a Categorical HMM
event_based = []
for music in my_dataset:
    event_based.append(muspy.to_event_representation(music))

flat = [seq.flatten().tolist() for seq in event_based]
event_sequence = [note for seq in flat for note in seq]
# print(event_sequence)

my_model = CategoricalHMM(n_components=hidden_states)
my_model.fit(np.array([event_sequence]))


CategoricalHMM(n_components=50, n_features=356,
               random_state=RandomState(MT19937) at 0x78A03CB5A140)

In [148]:
#Sampling a sequence of music from the model and save as a MIDI file, name it "event_based.mid"
sampled_event_seq = my_model.sample(n_samples=sythetic_music_sequence_length)
# print(sampled_event_seq)
synthetic_music = muspy.from_event_representation(np.array(sampled_event_seq[0]), resolution=24, program=0, is_drum=False, use_single_note_off_event=False, use_end_of_sequence_event=False, max_time_shift=100, velocity_bins=32, default_velocity=64, duplicate_note_mode='fifo')
# print(synthetic_music)
muspy.write_midi('event_based.mid', synthetic_music) #Specify the path to save the MIDI file, name it "event_based.mid"

To add some fun, lets take it up a notch and go for the __Note Based Representation__.  
More on that here:
1. https://muspy.readthedocs.io/en/v0.3.0/representations/note.html
2. https://salu133445.github.io/muspy/classes/note.html

This is a bit tricky since we have 4 features per observation. 

In [149]:
#Write your code here and save the sampled sequence as MIDI file, name it "note_based.mid"
sythetic_music_sequence_length = 200 #Length of the synthetic music sequence to be generated (could be either a time step, an event or a note)
# Write your code here by fitting and generating from a Categorical HMM
note_based = []
for music in my_dataset:
    note_based.append(muspy.to_note_representation(music))

# flat = [seq.flatten().tolist() for seq in note_based]
note_sequence = [note for seq in note_based for note in seq]
# print(note_sequence)
print(note_sequence)

[array([ 0, 51, 12, 64]), array([ 0, 67, 24, 64]), array([ 0, 70, 24, 64]), array([12, 39, 12, 64]), array([24, 68, 12, 64]), array([24, 72, 12, 64]), array([36, 65, 12, 64]), array([36, 68, 12, 64]), array([48, 51, 12, 64]), array([48, 63, 12, 64]), array([48, 67, 24, 64]), array([60, 39, 12, 64]), array([60, 58, 12, 64]), array([72, 60, 12, 64]), array([72, 68, 12, 64]), array([84, 56, 12, 64]), array([84, 65, 12, 64]), array([96, 51, 12, 64]), array([96, 55, 36, 64]), array([96, 63, 24, 64]), array([108,  39,  12,  64]), array([120,  39,  12,  64]), array([120,  62,  12,  64]), array([132,  39,  12,  64]), array([132,  56,  12,  64]), array([132,  60,  12,  64]), array([144,  39,  12,  64]), array([144,  55,  12,  64]), array([144,  58,  12,  64]), array([156,  51,  12,  64]), array([156,  55,  12,  64]), array([156,  58,  12,  64]), array([168,  50,  12,  64]), array([168,  53,  12,  64]), array([168,  58,  12,  64]), array([180,  48,  12,  64]), array([180,  51,  12,  64]), array(

In [150]:
from hmmlearn.hmm import GaussianHMM, GMMHMM

my_model = GaussianHMM(n_components=hidden_states)
my_model.fit(np.array(note_sequence))

GaussianHMM(n_components=50)

In [151]:
sampled_note_seq = my_model.sample(n_samples=sythetic_music_sequence_length)
sampled_note_seq = sampled_note_seq[0].astype(int)
# print(sampled_note_seq)

synthetic_music = muspy.from_note_representation(np.array(sampled_note_seq), resolution= 24, program= 0, is_drum= False, use_start_end = False, encode_velocity = True, default_velocity = 64 )

print(synthetic_music)


synthetic_music.tracks[0].notes[0].time = 0
for i in range(1,len(synthetic_music.tracks[0].notes)):
  synthetic_music.tracks[0].notes[i].time = synthetic_music.tracks[0].notes[i-1].time + synthetic_music.tracks[0].notes[i-1].duration

print(synthetic_music)



Music(metadata=Metadata(schema_version='0.2'), resolution=24, tracks=[Track(program=0, is_drum=False, notes=[Note(time=-354, pitch=40, duration=26, velocity=64), Note(time=-323, pitch=69, duration=11, velocity=64), Note(time=-310, pitch=54, duration=18, velocity=63), ...])])
Music(metadata=Metadata(schema_version='0.2'), resolution=24, tracks=[Track(program=0, is_drum=False, notes=[Note(time=0, pitch=40, duration=26, velocity=64), Note(time=26, pitch=69, duration=11, velocity=64), Note(time=37, pitch=54, duration=18, velocity=63), ...])])


In [152]:
muspy.write_midi('note_based.mid', synthetic_music)